In [ ]:
from project import Projection
from Problem import Problem
from main_dynamic import run_experiment, load_config
from Phase1 import solve_CQ_feasible
from Phase2_1_obj import optim_Universal
from utils import visualize_pareto_front

In [ ]:
import autograd.numpy as np
from autograd import jacobian
from scipy.optimize import Bounds
import matplotlib.pyplot as plt
import pandas as pd
import math

# Prob

In [ ]:
def f(x):
    return np.array([
        4*x[0]**2 + 4*x[1]**2,
        (x[0] - 5)**2 + (x[1] - 5)**2
    ])


#--------------- C --------------------#
def c1(x):
    return -(x[0] - 5)**2 - x[1]**2 + 25
def c2(x):
    return -(x[0] - 8)**2 - (x[1] + 3)**2 + 17.7


#--------------- Q --------------------#
def q1(y):
    return 50**2 - (y[0] - 50)**2 - (y[1] - 50)**2

def q_plus(y):
    center = 50.0
    radius_sq = 50**2  
    
    dx = np.maximum(0, y[0] - center)
    dy = np.maximum(0, y[1] - center)
    return radius_sq - (dx**2 + dy**2)

# Hàm dùng cho Projection 
cons_C = (
    {
        'type': 'ineq', 'fun' : c1,     
    },
    {
        'type': 'ineq', 'fun' : c2,     
    }
)

dim_x = 2
bounds_x = Bounds([-15,-15],[30, 30])

cons_Q = (
    {
        'type': 'ineq', 'fun' : q1,     
    },    
)

cons_Qplus = (
    {'type': 'ineq', 'fun': q_plus},
)
dim_y = 2

In [ ]:
# Setup Projections
proj_C_handler = Projection(cons=cons_C, bounds=bounds_x, dim=dim_x, proj_type='euclid')
proj_Q_handler = Projection(cons=cons_Q, bounds=None, dim=dim_y, proj_type='qplus')

# Setup Problem
prob = Problem(
    f=[f], jac_f=[jacobian(f)], C=[], Q=[], 
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)

In [ ]:
x_init = np.array([-10., -10.])

# Config

In [ ]:
from ruamel.yaml import YAML
from ruamel.yaml.comments import CommentedSeq

def update_config(file_path, x_init_np):
    yaml = YAML()
    yaml.preserve_quotes = True
    yaml.indent(mapping=2, sequence=4, offset=2)

    with open(file_path, 'r', encoding='utf-8') as f_in:
        config = yaml.load(f_in)

    val_list = x_init_np.tolist() if hasattr(x_init_np, 'tolist') else x_init_np
    
    flow_list = CommentedSeq(val_list)
    flow_list.fa.set_flow_style() 

    config['initialization']['x_init'] = flow_list
    
    config['phase1']['gamma'] = 0.00055
    config['phase1']['max_iter'] = 300
    
    config['phase2']['mu'] = 1
    config['phase2']['init_params'] = 1
    config['phase2']['expo_alpha'] = 0.25
    config['phase2']['expo_lambda'] = 0.75
    config['phase2']['max_iter'] = 500

    # 3. Ghi lại vào file
    with open(file_path, 'w', encoding='utf-8') as f_in:
        yaml.dump(config, f_in)
    
    print(f"✅ Đã cập nhật xong file: {file_path}")

# --- CHẠY THỬ ---
config_file = 'config.yaml'
update_config(config_file, x_init)

with open(config_file, 'r', encoding='utf-8') as f_in:
    print("\nNội dung file sau khi sửa:")
    print(f_in.read())

# Solve

In [ ]:
results = run_experiment(prob)

In [ ]:
visualize_pareto_front(
    pf_pred=results["pareto_f"], 
    pf_cloud=pf_cloud,   
    pf_targets=pf_targets,
    title="Objective Space Visualization",
    figsize=(12, 8)
)

In [8]:
!jupyter nbconvert --to html Ex4_truth_dynamic.ipynb

Traceback (most recent call last):
  File "D:\anaconda_env\my_env\lib\site-packages\traitlets\utils\importstring.py", line 35, in import_item
    pak = getattr(module, obj)
AttributeError: module 'jupyter_contrib_nbextensions.nbconvert_support' has no attribute 'CodeFoldingPreprocessor'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "D:\anaconda_env\my_env\Scripts\jupyter-nbconvert-script.py", line 9, in <module>
    sys.exit(main())
  File "D:\anaconda_env\my_env\lib\site-packages\jupyter_core\application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "D:\anaconda_env\my_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "D:\anaconda_env\my_env\lib\site-packages\nbconvert\nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "D:\anaconda_env\my_env\lib\site-packages\nbconvert\nbconvertapp.py", line 586